In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
#import cvxpy
import math
import numpy as np
#import cubic_spline_planner

N_IND_SEARCH = 10  # Search index number
show_animation = True

!rm -f Data_Processor.pyc
Remove=True
if Remove==True:
    !rm -r Fig_iLQR
    !rm -r Rec_iLQR
    !rm -r Rec_iLQR_Raw

In [2]:
import numpy as np
import math
from copy import copy, deepcopy
TARGET_SPEED = 20# / 3.6  # [m/s] target speed
MAX_TIME=1500
T=5
NX=4
NU=2
DT=0.2
WB=50
MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 25#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -25#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 5.0  # maximum accel [m/ss]
    


In [3]:
from Motion import Motion,State
R = np.diag([0.5, 10.0])  # input cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix


def np_quad_form(a,b):
        res=a @ b @ a.T
        return res
    
    
    
def sim_n_steps(_x0,U,xref,N):
    _U=U.copy()
    X=np.zeros((NX,N+1),dtype=np.float64)
    X[:,0]=_x0
    
    total_grad=0
    loss=0
    for t in range(N):
        x_t=X[:,t]
        u_t=_U[:,t]
        xref_t=xref[:,t]
        x_t1=sim_1_step(x_t,u_t)
        loss += np_quad_form(u_t, R) + np_quad_form(xref_t - x_t, Q)
        #cc=calc_cost(x_t1,u_t,xref_t,t)
        X[:,t+1]=x_t1.copy()
    return X,loss,U

def sim_1_step(_X,_U):
    
    _state=State(x=_X[0],y=_X[1],v=_X[2],yaw=_X[3])
    nst,Const=Motion.get_next_state(_state,_U[0],_U[1])
    x_t1=nst.array()
    return x_t1#np.array(x_t1,dtype=np.float64)

In [4]:
class iLQRController():
    def __init__(self):
        self.STATE_SIZE=NX
        self.INPUT_SIZE=NU
        self.max_iter=10
        self.dt=0.2
        self.lamb_factor=1.0
        self.lamb_max=10e4
        self.tN=T
        self.eps_converge=0.001
        
    def init_forward_matrix(self):
        self.f_x = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # df / dx
        self.f_u = np.zeros((self.tN, self.STATE_SIZE, self.INPUT_SIZE)) # df / du
        
    def init_cost(self):
        self.l = np.zeros((self.tN,1)) # immediate state cost 
        self.l_x = np.zeros((self.tN, self.STATE_SIZE)) # dl / dx
        self.l_xx = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # d^2 l / dx^2
        self.l_u = np.zeros((self.tN, self.INPUT_SIZE)) # dl / du
        self.l_uu = np.zeros((self.tN, self.INPUT_SIZE, self.INPUT_SIZE)) # d^2 l / du^2
        self.l_ux = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # d^2 l / du / dx
    
    def calc_cost(self, _x, _u, _xref,t):
        
        
        self.l[t] = np_quad_form(_u, R) + np_quad_form(_xref - _x, Q)
        
        loss_x1=2*Q[0,0]*(_xref[0]-_x[0])*(-1)
        loss_x2=2*Q[1,1]*(_xref[1]-_x[1])*(-1)
        loss_x3=2*Q[2,2]*(_xref[2]-_x[2])*(-1)
        loss_x4=2*Q[3,3]*(_xref[3]-_x[3])*(-1)
            
        self.l_x[t]=np.array([loss_x1,loss_x2,loss_x3,loss_x4])
        self.l_xx[t]=2 * Q
        
        Qdiag=np.array([Q[0,0], Q[1,1], Q[2,2], Q[3,3]],dtype=np.float64)
        error=np.array(_xref-_x)
        
        loss_u1 = 2. * _u[0] * R[0,0] #+ np.sum(2*Qdiag*error*(-1)*_B[:,0])
        loss_u2 = 2. * _u[1] * R[1,1] #+ np.sum(2*Qdiag*error*(-1)*_B[:,1])
        
        self.l_u[t] = np.array([loss_u1, loss_u2])
        self.l_uu[t] = 2. * R
        self.l_ux[t] = np.zeros((NU, NX))
    
    def init_ks(self):
        self.k = np.zeros((self.tN, self.INPUT_SIZE)) # feedforward modification
        self.K = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # feedback gain

  
    def calc_forward_matrix(self,_X,_U,t):
        A, B, C = Motion.get_linear_model_matrix(_X[2],_X[3], _U[1])
        self.f_x[t] = A
        self.f_u[t] = B 
        
    def ilqr(self, x0, U,xref): 
        
        lamb = 1.0 # regularization parameter
        sim_new_trajectory = True
        tN = U.shape[0] # number of time steps
        X=np.zeros((NX,T+1),dtype=np.float64)
        X[:,0]=x0.copy()
        for ii in range(self.max_iter):

            if sim_new_trajectory == True: 
                #X, cost, _ = sim_n_steps(x0, U,xref,T)
                #oldcost = copy(cost) # copy for exit condition check

                
                self.init_forward_matrix()
                self.init_cost()
                for t in range(self.tN):
                    x_t=X[:,t].copy()
                    u_t=U[:,t].copy()
                    xref_t=xref[:,t].copy()
                    self.calc_forward_matrix(x_t,u_t,t)
                    self.calc_cost(x_t, u_t,xref_t,t)
                    X[:,t+1]=sim_1_step(x_t,u_t)
                _cost =np.sum(self.l)
                sim_new_trajectory = False

            V = self.l[-1].copy()  
            V_x = self.l_x[-1].copy() 
            V_xx = self.l_xx[-1].copy() 
            self.init_ks()
 
            for t in range(self.tN-2, -1, -1):
                
                Q_x = self.l_x[t] + np.dot(self.f_x[t].T, V_x) 
                Q_u = self.l_u[t] + np.dot(self.f_u[t].T, V_x)

                Q_xx = self.l_xx[t] + np.dot(self.f_x[t].T, np.dot(V_xx, self.f_x[t])) 
                Q_ux = self.l_ux[t] + np.dot(self.f_u[t].T, np.dot(V_xx, self.f_x[t]))
                Q_uu = self.l_uu[t] + np.dot(self.f_u[t].T, np.dot(V_xx, self.f_u[t]))

                Q_uu_evals, Q_uu_evecs = np.linalg.eig(Q_uu)
                Q_uu_evals[Q_uu_evals < 0] = 0.0
                #Q_uu_evals += lamb
                Q_uu_inv = np.dot(Q_uu_evecs, np.dot(np.diag(1.0/Q_uu_evals), Q_uu_evecs.T))
                #k[t]=-1*l_u[t]
                self.k[t] = -1. * np.dot(Q_uu_inv, Q_u)
                self.K[t] = -1. * np.dot(Q_uu_inv, Q_ux)

                V_x = Q_x - np.dot(self.K[t].T, np.dot(Q_uu, self.k[t]))
                V_xx = Q_xx - np.dot(self.K[t].T, np.dot(Q_uu, self.K[t]))

            U_new = np.zeros((NU,T),dtype=np.float64)
            x_new = np.zeros((NX,T),dtype=np.float64)
            x_new = x0.copy()
            for t in range(self.tN - 1): 
               
                U_new[:,t] = U[:,t] + self.k[t] + np.dot(self.K[t], x_new - X[:,t])
                x_new_1= sim_1_step(x_new, U_new[:,t])
                x_new=x_new_1.copy()
            # evaluate the new trajectory 
            X_new, _new_cost , _= sim_n_steps(x0, U_new,xref,T)
            #if 1==1:
      
            if _new_cost < _cost: 
                # decrease lambda --> get closer to Newton's method
                lamb /= self.lamb_factor

                X = copy(X_new) 
                U = copy(U_new) 
                #oldcost = copy(cost)
                #cost = copy(cost_new)

                sim_new_trajectory = True 

                #print('...',_cost,_new_cost)
                if ii > 0 and ((abs(_cost-_new_cost)/_new_cost) < self.eps_converge):
                    print("break 1",ii)
                    break

            else: 
                # increase lambda --> closer to gradient descent
                lamb *= self.lamb_factor
                if lamb > self.lamb_max: 
                    print("break 2",ii)
                    break
        print('return ii',ii,_new_cost,_cost)
        return X, U, _cost
    

In [5]:
import os
from Car_Env import Car_Env
from Data_Processor import Data_Processor
from Car_Plot import Move_Contour,Draw_Car

In [6]:
#!rm -r Exp2_456

In [7]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
rec=True
max_len=350
qlen=25
car_move=True
seed=0
root='Exp3_'+str(seed)+'/'
if not os.path.exists(root):
        os.mkdir(root)
        
np.random.seed(seed)
Env=Car_Env(rec=rec,max_len=max_len,qlen=qlen,_path=root,seed=seed)
[init_state,map_points]=Env.reset()
mpx=np.array(map_points[:max_len,0])#-ofx
mpy=np.array(map_points[:max_len,1])#-ofy
ix,iy,iyaw,ivel=init_state[0]
_traj=init_state[1][0]
_dind=init_state[1][1]
_end=init_state[1][2]
ofx,ofy,ofang=init_state[2]
img=init_state[3]
state = State(x=ix, y=iy, yaw=iyaw, v=ivel)
#print(init_state)

Track generation: 1143..1442 -> 299-tiles track


In [8]:
if rec==True:
    if not os.path.exists(root+'Rec_iLQR/'):
        os.mkdir(root+'Rec_iLQR')
    if not os.path.exists(root+'Rec_iLQR_Raw/'):
        os.mkdir(root+'Rec_iLQR_Raw')
    if not os.path.exists(root+'Fig_iLQR/'):
        os.mkdir(root+'Fig_iLQR')


In [ ]:
import cv2
import matplotlib.gridspec as gridspec
import time
phi=0
rot_mat=np.array([[math.cos(phi),math.sin(phi)],[-math.sin(phi),math.cos(phi)]])
dis_time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]

log_state=[]
log_traj=[]
log_img=[]
log_act=[]
log_next_state=[]
log_reward=[]


#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
_index=0
#cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''


_Data_Processor=Data_Processor(_speed=TARGET_SPEED,tlen=qlen)
acc_st=State(x=0,y=0)
itr=0
#fig,axes=plt.subplots(nrows=2,ncols=1,figsize=(10,10))
fig=plt.figure(figsize=(10,10))
gs = fig.add_gridspec(3, 1)
ax1=fig.add_subplot(gs[0,:])
ax2=fig.add_subplot(gs[1:,:])
axes=[ax1,ax2]


def map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _ofang=-1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)

    _vect[0]+=_ofx
    _vect[1]+=_ofy
    return [_vect[0],_vect[1]]
def inv_map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _vect[0]-=_ofx
    _vect[1]-=_ofy
    _ofang=1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)
    return [_vect[0],_vect[1]]
U=np.zeros((NU,T),dtype=np.float64)
iLQR=iLQRController()
init_time=time.time()
while MAX_TIME >= dis_time:
       
        _full_state = np.array([state.x, state.y, state.v, state.yaw],dtype=np.float64)
    
        
        _full_state_traj=_Data_Processor.update_trajectory(_traj)
        xref, dref = _Data_Processor.get_ref_trajectory(state,dl,DT,T)
        
        if _end==True:
            break

        _index+=_dind
        
        #oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(xref, Motion_Status, dref, oa, odelta)
        X,U, cost= iLQR.ilqr(_full_state, U, xref )
        oa=U[0,1]
        odelta=U[1,1]
        ox=X[0,1]
        oy=X[1,1]
        ov=X[2,1]
        oyaw=X[3,1]
        state_str=' x: '+str('%0.4f'%state.x)+' y: '+str('%0.4f'%state.y)+' v: '+str('%0.4f'%state.v)+' yaw: '+str('%0.4f'%np.rad2deg(state.yaw))+'\n'
        xref_str =' x: '+str('%0.4f'%xref[0][0])+' y: '+str('%0.4f'%xref[1][0])+' v: '+str('%0.4f'%xref[2][0])+' yaw: '+str('%0.4f'%np.rad2deg(xref[3][0]))+'\n'
        time_str ='Time(Sec): '+str('%0.4f'%dis_time)+' Sec Distance(m): '+str('%0.4f'%_index)+' m '+'\n'
        #xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        act_str  =' accel: '+str('%0.4f'%accl)+' gas: '+str('%0.4f'%gas)+' steer: '+str('%0.4f'%np.rad2deg(steer))+'\n'
    
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        max_steer=np.deg2rad(45)
        max_accl=5
        if odelta is not None:
            if abs(U[1,0])<max_steer:
                  di= U[1,0]
            else:
                di=(U[1,0]/abs(U[1,0]))*max_steer
            
            if abs(U[0,0])<max_accl:
                  ai=U[0,0]
            else:
                ai=(U[0,0]/abs(U[0,0]))*max_accl
        accl=ai
        steer=di
        
        #act_str=str(steer)+'\n'
        _log_state=[ofx,ofy,_full_state[0],ofang]
        nst,path,offset,img,rw,ter,info=Env.step([steer,accl,0])
        img_raw=Env.render(mode='rgb_array')
        _traj,_dind,_end=path
        ofx,ofy,ofang=offset
        
        gas=Env.gas
        
        state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        _log_next_state=[ofx,ofy,state.v,ofang]
        #Motion_Status.set_state(next_state)
        curr_time=time.time()-init_time
        log_state.append(_log_state)
        log_traj.append(_full_state_traj)
        log_img.append(img)
        log_act.append([accl,steer,0])
        log_next_state.append(_log_next_state)
        log_reward.append([dis_time,curr_time,_index,rw,ter])
        
        dis_time = dis_time + DT
        if car_move==True:
            _xref=xref.copy()
            [_ox,_oy]=map_plot([ox,oy],ofx,ofy,ofang)
            [_xref[0,:],_xref[1,:]]=map_plot([xref[0,:],xref[1,:]],ofx,ofy,ofang)
            [_x,_y]=map_plot([state.x,state.y],ofx,ofy,ofang)
            car_x,car_y,car_ang=ofx,ofy,ofang
            _mpx,_mpy=mpx,mpy
        else:
            _ox,_oy=ox,oy
            _xref=xref
            _x,_y=state.x,state.y
            car_x,car_y,car_ang=state.x,state.y,state.yaw
            [_mpx,_mpy]=inv_map_plot([mpx,mpy],ofx,ofy,ofang)
        x.append(_x)
        y.append(_y)
        yaw.append(ofang)
        v.append(state.v)
        t.append(dis_time)
        d.append(di)
        a.append(ai)

        
    
      
        
        if show_animation:  # pragma: no cover
            axes[0].cla()
            axes[1].cla()
            axes[0].text(.01,.6,'Running : '+time_str,fontsize=10,color='c')
            axes[0].text(.01,.4,'state : '+state_str,fontsize=10,color='r')
            axes[0].text(.01,.2,'xref : '+xref_str,fontsize=10,color='b')
            axes[0].text(.01,0,'act : '+act_str,fontsize=10,color='g')
            
            if ox is not None:
                axes[1].plot(_ox, _oy, "xr", label="MPC",alpha=1.0)
            axes[1].plot(_mpx, _mpy, "-b", label="course")
            axes[1].plot(x, y, "oc", label="trajectory")
            axes[1].plot(_xref[0, :], _xref[1, :], "xk", label="xref",alpha=1.0)
            axes[1].plot(_mpx[_index], _mpy[_index], "xg", label="target",alpha=1.0)
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            car_points=Draw_Car(car_x,car_y)
            rp=Move_Contour(car_points,0,0,car_ang)
            r_car_points=rp.apply()
            r_points=np.array(r_car_points,dtype=np.float64)
            axes[1].plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            #plt.title("Time[s]:" + str(round(time, 2))
            #          + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
            #          + '\nyaw'+str(state.yaw)
            #          + '\nxref_yaw'+str(xref[3][0])
            #          + '\nsteer'+str(steer))
            #fig.canvas.draw()
            plt.pause(0.0001)
            if rec==True:
                if itr%1==0:
                    plt.savefig(root+'Fig_iLQR/'+str(int(itr/1))+'.jpg')
                    cv2.imwrite(root+'Rec_iLQR/'+str(int(itr/1))+'.jpg',img)
                    cv2.imwrite(root+'Rec_iLQR_Raw/'+str(int(itr/1))+'.jpg',img_raw)
            itr+=1
            if ter==True:
                print('terminal ... ')
                
if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(car_x, car_y, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


break 1 1
return ii 1 1347.4669532374241 1347.4884107025425
break 1 1
return ii 1 1324.8887146417926 1324.888715573045
break 1 1
return ii 1 1307.21986600334 1307.2198728631229
break 1 1
return ii 1 1289.0119591195817 1289.011967643604
break 1 1
return ii 1 1271.002671862755 1271.0026806518385
break 1 1
return ii 1 1252.921985676439 1252.921995063314
break 1 1
return ii 1 1234.7434068362795 1234.7434167972929
break 1 1
return ii 1 1216.430963314401 1216.4309739040011
break 1 1
return ii 1 1197.9713590326935 1197.971370232409
break 1 1
return ii 1 1179.3587283725599 1179.3587401520447
break 1 1
return ii 1 1160.5938770304929 1160.5938893199504
break 1 1
return ii 1 1141.6808759020075 1141.6808886036742
break 1 1
return ii 1 1122.6265782030362 1122.62659118628
break 1 1
return ii 1 1103.438958064028 1103.4389711643012
break 1 1
return ii 1 1084.1280213328587 1084.1280343563067
break 1 1
return ii 1 1064.7026035183474 1064.7026162467168
break 1 1
return ii 1 1045.1737374957659 1045.173749

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\irfan\Anaconda3\envs\work\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\irfan\Anaconda3\envs\work\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "C:\Users\irfan\Anaconda3\envs\work\lib\site-packages\matplotlib\backends\_backend_tk.py", line 338, in idle_draw
    self.draw()
  File "C:\Users\irfan\Anaconda3\envs\work\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 9, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "C:\Users\irfan\Anaconda3\envs\work\lib\site-packages\matplotlib\backends\backend_agg.py", line 388, in draw
    self.figure.draw(self.renderer)
  File "C:\Users\irfan\Anaconda3\envs\work\lib\site-packages\matplotlib\artist.py", line 38, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\Users\irfan\Anaconda3\envs\work\lib\site-packages\matplotlib\figure.py", line 1709, in draw


break 1 1
return ii 1 153.0458353280397 153.04583543247733
break 1 1
return ii 1 141.63963860509284 141.6396386987513
break 1 1
return ii 1 131.1924097956324 131.19240986507864
break 1 1
return ii 1 121.73231131778863 121.73231136414209
break 1 1
return ii 1 113.28782552630088 113.28782555392345
break 1 1
return ii 1 105.8878741218679 105.8878741364207
break 1 1
return ii 1 99.70349705169203 99.70349705833718
break 1 1
return ii 1 94.86240737034386 94.86240737289558


In [ ]:
log_dict={
'state':log_state,
'traj':log_traj,
'image':log_img,
'action':log_act,
'next_sate':log_next_state,
'time_reward':log_reward
}

In [ ]:
import pickle
f=open(root+'dataset_iLQR','wb')
pickle.dump(log_dict,f)
f.close()
itr

In [ ]:
f=open('dataset_iLQR','rb')
dataset=pickle.load(f)
f.close()

In [ ]:
dataset.keys()

In [ ]:
traj=np.array(log_traj)#np.array(dataset['traj'])
plt.plot(traj[])
plt.show()

In [ ]:
traj[200]